In [ ]:
import json
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

def pdf_retriever_function(query):
    """Function to retrieve information from PDF"""
    return f"Retrieved information for query: {query}"

# Define tools
tools = [
    Tool(
        name="PDF_Search",
        func=pdf_retriever_function,
        description="Search through the PDF document"
    )
]

# Updated prompt template to enforce strict JSON formatting
template = """Answer the following question as best you can.

Question: {input}

IMPORTANT: Your response must be a valid JSON object with exactly this format:
{{
    "answer": "your concise answer here",
    "explanation": "your detailed explanation here"
}}

Do not include any text outside of this JSON structure. Make sure to escape any quotes within your answer and explanation.

{agent_scratchpad}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input", "agent_scratchpad"]
)

# Initialize the LLM
llm = Ollama(model="deepseek-llm:7b-chat")

# Create the chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Create the agent with handle_parsing_errors enabled
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True
)

# Create the agent executor with parsing error handling
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True  # Add this to handle parsing errors
)

def load_pdf_and_create_vector_store(pdf_path):
    """Load PDF and create vector store"""
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    embeddings = OllamaEmbeddings(model="deepseek-llm:7b-chat")
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

def query_agent(question: str):
    """Query the agent with a question"""
    try:
        # Execute the query
        response = agent_executor({"input": question})
        
        # If the response is a string (raw LLM output), try to format it as JSON
        if isinstance(response, str):
            try:
                # Try to extract just the JSON part if there's extra text
                json_start = response.find('{')
                json_end = response.rfind('}') + 1
                if json_start >= 0 and json_end > 0:
                    json_str = response[json_start:json_end]
                    return json.loads(json_str)
                else:
                    # If no JSON found, format the entire response
                    return {
                        "answer": response[:500],  # Truncate long answers
                        "explanation": "Response was formatted as plain text"
                    }
            except json.JSONDecodeError:
                # If JSON parsing fails, return the response in our format
                return {
                    "answer": response[:500],  # Truncate long answers
                    "explanation": "Full response in explanation"
                }
        
        return response
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            "answer": "Error occurred while processing the query",
            "explanation": str(e)
        }

# Example usage
if __name__ == "__main__":
    # Initialize vector store
    pdf_path = "English.pdf"
    vectorstore = load_pdf_and_create_vector_store(pdf_path)
    
    # Test the system
    query = "What is the role of counselling in preventing drug addiction?"
    result = query_agent(query)
    print(json.dumps(result, indent=2))

/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykernel_1991/2922089001.py:169: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykernel_1991/2922089001.py:175: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykernel_1991/2922089001.py:178: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more fl



> Entering new AgentExecutor chain...
The role of counselling in preventing drug addiction is to provide guidance, support and education on healthy decision-making and coping strategies. Counselling can help individuals identify potential triggers for drug use, develop problem-solving skills, manage emotions and stress effectively, and build resilience against the pressures that may lead them to abuse drugs.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

> Finished chain.
{
  "input": "What is the role of counselling in preventing drug addiction?",
  "output": "Agent stopped due to iteration limit or time limit."
}
